<a href="https://colab.research.google.com/github/sitikanthaprusty2904/DL_step_in/blob/main/hyper_para.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd


In [12]:
df=pd.read_csv('/content/diabetes[1].csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [14]:

X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [15]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)
# y=sc.fit_transform(y.reshape(-1,1)) # Removed scaling for y

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout


In [198]:
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
# model.add(Dense(16,activation='relu'))
# model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [199]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [200]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5735 - loss: 0.6947 - val_accuracy: 0.7078 - val_loss: 0.6291
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6907 - loss: 0.6323 - val_accuracy: 0.7403 - val_loss: 0.5978
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7306 - loss: 0.5948 - val_accuracy: 0.7532 - val_loss: 0.5727
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7519 - loss: 0.5695 - val_accuracy: 0.7597 - val_loss: 0.5552
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7386 - loss: 0.5570 - val_accuracy: 0.7727 - val_loss: 0.5394
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7678 - loss: 0.5180 - val_accuracy: 0.7727 - val_loss: 0.5279
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7876 - loss: 0.5030 - val_accuracy: 0.7727 - val_loss: 0.5179
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7761 - loss: 0.5000 - val_accuracy: 0.7792 - 

In [201]:
#  HYPERPARAMETER TUNING
# select optimizer
# no of nodes in a layer
# no of layers
# All in one

In [4]:
!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.1 MB/s eta 0:00:00


In [5]:
import kerastuner as kt

/tmp/ipython-input-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [204]:
from keras.src.optimizers import optimizer
def model_build(hp):
  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  # model.add(Dense(16,activation='relu'))
  # model.add(Dense(8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [205]:
tuner=kt.RandomSearch(
    model_build,
    objective='val_accuracy',
    max_trials=5,
)

Reloading Tuner from ./untitled_project/tuner0.json


In [206]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))
# hyperparammeter tuning (trials fro optimiser)

In [207]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [208]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [209]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [210]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7247 - loss: 0.5266 - val_accuracy: 0.7013 - val_loss: 0.5449
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7625 - loss: 0.4973 - val_accuracy: 0.7078 - val_loss: 0.5367
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7478 - loss: 0.4917 - val_accuracy: 0.7208 - val_loss: 0.5322
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7587 - loss: 0.4564 - val_accuracy: 0.7208 - val_loss: 0.5282
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7724 - loss: 0.4509 - val_accuracy: 0.7208 - val_loss: 0.5252
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7454 - loss: 0.4846 - val_accuracy: 0.7143 - val_loss: 0.5232
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7706 - loss: 0.4614 - val_accuracy: 0.7403 - val_loss: 0.5226
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7707 - loss: 0.4684 - val_accuracy: 0.73

NO OF NEURON IN A LAYER TUNING

In [217]:
def model_build2(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=256,step=8)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [223]:
tuner_neurons=kt.RandomSearch(
    model_build2,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name='myproject'
)

Reloading Tuner from mydir/myproject/tuner0.json


In [228]:
tuner_neurons.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [225]:
tuner_neurons.get_best_hyperparameters()[0].values

{'units': 48}

In [226]:
model = tuner_neurons.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [227]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7256 - loss: 0.5633 - val_accuracy: 0.7792 - val_loss: 0.5088
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7803 - loss: 0.5025 - val_accuracy: 0.7922 - val_loss: 0.4996
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7543 - loss: 0.5009 - val_accuracy: 0.7922 - val_loss: 0.4954
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7828 - loss: 0.5012 - val_accuracy: 0.7922 - val_loss: 0.4915
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7774 - loss: 0.4913 - val_accuracy: 0.7987 - val_loss: 0.4908
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8046 - loss: 0.4482 - val_accuracy: 0.8052 - val_loss: 0.4903
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7947 - loss: 0.4609 - val_accuracy: 0.7922 - val_loss: 0.4897
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8073 - loss: 0.4428 - val_accuracy: 0.7857 - 

No of layers

In [234]:
def build_model(hp):
  model=Sequential()
  l=hp.Int('units',min_value=1,max_value=10)
  model.add(Dense(48,activation='relu',input_dim=8))
  for i in range(l):
    model.add(Dense(48,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [235]:
tuner_layer=kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir',
    project_name='myproject2'
)

Reloading Tuner from mydir/myproject2/tuner0.json


In [236]:
tuner_layer.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 01m 48s


In [237]:
tuner_layer.get_best_hyperparameters()[0].values

{'units': 1}

In [239]:
model=tuner_layer.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 48)             │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 48)             │         2,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            49 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,833 (11.07 KB)

 Trainable params: 2,833 (11.07 KB)

 Non-trainable params: 0 (0.00 B)

In [240]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test) ,batch_size=48)

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.7784 - loss: 0.4794 - val_accuracy: 0.7792 - val_loss: 0.5035
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7775 - loss: 0.4782 - val_accuracy: 0.7922 - val_loss: 0.5035
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7594 - loss: 0.4861 - val_accuracy: 0.7792 - val_loss: 0.5049
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7809 - loss: 0.4555 - val_accuracy: 0.7727 - val_loss: 0.5080
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7866 - loss: 0.4402 - val_accuracy: 0.7468 - val_loss: 0.5129
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7927 - loss: 0.4416 - val_accuracy: 0.7597 - val_loss: 0.5146
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7890 - loss: 0.4406 - val_accuracy: 0.7727 - val_loss: 0.5151
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7733 - loss: 0.4510 - val_accuracy: 0.7727 -

ALL  IN  ONE

In [38]:
def model_all(hp):
  model=Sequential()
  model.add(Dense(hp.Int('units',min_value=8,max_value=256,step=8),
                  activation=hp.Choice('activation',values=['relu','tanh','sigmoid']),
                  input_dim=8))
  for i in range(hp.Int('units_',min_value=1,max_value=10)):
    model.add(Dense(
        hp.Int('units'+str(i),min_value=8,max_value=256,step=8),
        activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop']),
                loss='binary_crossentropy',metrics=['accuracy']
                )
  return model



In [39]:
tuner_f=kt.RandomSearch(
    model_all,
    objective='val_accuracy',
    max_trials=5,
    directory='mydir_f',
    project_name='myproject_f'

)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [40]:
tuner_f.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 33s


In [41]:
tuner_f.get_best_hyperparameters()[0].values

{'units': 192,
 'activation': 'sigmoid',
 'units_': 2,
 'units0': 216,
 'activation0': 'relu',
 'optimizer': 'rmsprop',
 'units1': 96,
 'activation1': 'tanh',
 'units2': 16,
 'activation2': 'relu',
 'units3': 216,
 'activation3': 'relu',
 'units4': 208,
 'activation4': 'tanh',
 'units5': 200,
 'activation5': 'relu',
 'units6': 88,
 'activation6': 'tanh',
 'units7': 88,
 'activation7': 'relu',
 'units8': 40,
 'activation8': 'relu'}

In [43]:
model=tuner_f.get_best_models(num_models=1)[0]
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 192)            │         1,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 216)            │        41,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        20,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            97 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,345 (251.35 KB)

 Trainable params: 64,345 (251.35 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
model.fit(X_train,y_train,epochs=200,validation_data=(X_test,y_test))

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.7378 - loss: 0.5437 - val_accuracy: 0.7532 - val_loss: 0.5305
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7672 - loss: 0.5019 - val_accuracy: 0.7273 - val_loss: 0.5041
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7627 - loss: 0.4915 - val_accuracy: 0.7273 - val_loss: 0.5111
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7397 - loss: 0.4961 - val_accuracy: 0.7078 - val_loss: 0.5276
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7471 - loss: 0.5011 - val_accuracy: 0.6558 - val_loss: 0.7089
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7412 - loss: 0.5524 - val_accuracy: 0.7792 - val_loss: 0.5028
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7554 - loss: 0.4818 - val_accuracy: 0.7597 - val_loss: 0.4942
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7560 - loss: 0.4867 - val_accuracy: 0.7078 